In [91]:
import seaborn
import pandas as pd
import numpy as np


In [92]:
titanic = seaborn.load_dataset('Titanic')
titanic.sample()

,survived,pclass,sex,age,sibsp,parch,fare,embarked,class,who,adult_male,deck,embark_town,alive,alone
823,1,3,female,27.0,0,1,12.475,S,Third,woman,False,E,Southampton,yes,False


# Task 1

In [93]:
# Split the target 'survived' off from the predictors

In [94]:
#__SOLUTION__
X = titanic.drop('survived', axis=1)
y = titanic['survived']

In [95]:
# convert the 'sex' column to a binary where 1 represents female

In [96]:
#__SOLUTION__

def sex_converter(sex_element):
    
    if sex_element == 'female':
        return 1
    else:
        return 0
    
    
X['sex'] = X['sex'].apply(sex_converter)
X.sex.value_counts()

0    577
1    314
Name: sex, dtype: int64

# Task 2

In [97]:
# Implement a train-test split using all default arguments and random_state=42

In [98]:
#__SOLUTION__
from sklearn.model_selection import train_test_split
X_train, X_test, y_train, y_test = train_test_split(X,y, random_state=42)

# Task 3

We will fit a logistic regression model using only `sex` and `fare`.  Remember that logistic regression uses regularization by default.  That means that scaling will have an effect on the scores. 

In [99]:
# Fit a standard scaler to train columns of interest, and transform the test


In [100]:
#__SOLUTION__
from sklearn.preprocessing import StandardScaler
ss = StandardScaler()
ss.fit(X_train[['sex', 'fare']])

X_train_sc = ss.transform(X_train[['sex', 'fare']])
X_test_sc = ss.transform(X_test[['sex', 'fare']])



In [101]:
# Instantiate a logistic regression model with the default arguments and random_state=42

# Fit on the scaled training data. 


In [102]:
#__SOLUTION__
from sklearn.linear_model import LogisticRegression

lr = LogisticRegression(random_state=42)
lr.fit(X_train_sc, y_train)

LogisticRegression(random_state=42)

# Task 4

The coefficients produced by fitting the model dictate the predictions that the model makes.

In [103]:
# Model predictions
y_hat = lr.predict(X_test_sc)

Under the hood, the model is applying a decision threshold to assign a 0 or 1 prediction. 
For the exercise below, apply a decision threshold of .5 to the predicted probabities, so that you can recreate 0 or 1 predictions stored in y_hat above.

In [104]:
# Convert these predicted probabilities to 0/1 predictions
y_hat_proba = lr.predict_proba(X_test_sc)

# Apply threshold here
y_hat_by_hand = None

In [105]:
#__SOLUTION__
# Convert these predicted probabilities to 0/1 predictions
y_hat_proba = lr.predict_proba(X_test_sc)

# Apply threshold here
y_hat_by_hand = (y_hat_proba[:,1] > .5)

In [106]:
assert (y_hat == y_hat_by_hand).sum() == len(y_hat)

# Stretch Task

For the final task, we will recreate the predicted predicted probabilities using the coefficients.

In [108]:
def sigmoid(log_odd):
    
    '''
    The link function translates a log_odd prediction
    and returns a probability of class 1 with a
    number between 0 and 1.
    '''
    
    return 1/(1+np.e**(-log_odd))
    

In the cell below, use the coef_ and intercept_ attributes from the fit model to calculate the log_odds for each record in the test set. Then pass these log_odds into the sigmoid function above.

In [109]:
# Log odds are the output of the dot product of X_test, 
# a version of the coef_ attribute, plus the interceps_ attribute. 
log_odds = None

# use a list comprehension to apply the sigmoid function to each log_odd.
predict_proba_by_hand = None

In [110]:
#__SOLUTION__
log_odds = X_test_sc.dot(lr.coef_.T) + lr.intercept_
predict_proba_by_hand = [sigmoid(log_odd)[0] for log_odd in log_odds]


In [111]:
assert np.isclose(predict_proba_by_hand, y_hat_proba[:,1]).sum() == 223